# Notebook 3: Injection Generation



In [1]:
# Built-in imports
import os
import sys
from typing import Iterator, List, Dict
from pathlib import Path
from itertools import islice

# Dependency imports: 
import numpy as np
import tensorflow as tf
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot

# GravyFlow import, again adding the grandparent directory to the path:

# Get the absolute path of the grandparent directory (two levels up from the current directory)
parent_dir : Path = os.path.abspath('../../')

# Add the grandparent directory to sys.path if it's not already there.
# This is necessary to import GravyFlow if it's located in the grandparent directory.
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

# Import the GravyFlow module.
import gravyflow as gf

2024-01-12 01:37:33.059356: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-12 01:37:52.905793: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Set up the environment using gf.env() and return a tf.distribute.Strategy object.
env : tf.distribute.Strategy = gf.env()

INFO:root:TensorFlow version: 2.12.1, CUDA version: 11.8
2024-01-12 01:38:50.803048: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2000 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:85:00.0, compute capability: 7.0
INFO:root:[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
phenom_d_generator : gf.WaveformGenerator = gf.cuPhenomDGenerator(
    mass_1_msun=50.0,
    mass_2_msun=50.0,
    inclination_radians=0.0
)

In [4]:
phenom_d_injection_generator : gf.InjectionGenerator = gf.InjectionGenerator(phenom_d_generator)

In [5]:
# Use the TensorFlow environment 'env' created earlier with gf.env()
with env:
    phenom_d_injection, _, _ = next(phenom_d_injection_generator(num_examples_per_batch=1))

2024-01-12 01:38:56.187047: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x559a58e016a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-01-12 01:38:56.187088: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): Tesla V100-SXM2-16GB, Compute Capability 7.0
2024-01-12 01:38:56.221677: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2024-01-12 01:38:56.245465: I ./tensorflow/compiler/jit/device_compiler.h:180] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


In [6]:
# Injection[injection_index][batch_index][polarization_index]
phenom_d_strain_plot = gf.generate_strain_plot(
    {"Plus Polarisation": phenom_d_injection[0][0][0],
     "Cross Polarisation": phenom_d_injection[0][0][1]
    },
    title="Phenom D Injection"
)

# Arrange the plots in a grid layout and display them in the notebook.
output_notebook()
show(phenom_d_strain_plot)

Loading BokehJS ...

In [7]:
wnb_generator : gf.WaveformGenerator = gf.WNBGenerator(
    duration_seconds=0.7,
    min_frequency_hertz=50,
    max_frequency_hertz=100.0
)

In [8]:
wnb_injection_generator : gf.InjectionGenerator = gf.InjectionGenerator(wnb_generator)

In [11]:
# Use the TensorFlow environment 'env' created earlier with gf.env()
with env:
    wnb_injection, _, _ = next(wnb_injection_generator(num_examples_per_batch=1))

2024-01-12 01:40:57.804492: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'num_waveforms' with dtype int32
	 [[{{node num_waveforms}}]]
2024-01-12 01:40:57.940388: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'foldl/while/hann_window/cond/ones/packed/foldl/while/TensorArrayV2Read/TensorListGetItem' with dtype int32
	 [[{{node foldl/while/hann_window/cond/ones/packed/foldl/while/TensorArrayV2Read/TensorListGetItem}}]]


In [12]:
# Injection[injection_index][batch_index][polarization_index]
wnb_strain_plot = gf.generate_strain_plot(
    {"Plus Polarisation": wnb_injection[0][0][0],
     "Cross Polarisation": wnb_injection[0][0][1]
    },
    title="Phenom D Injection"
)

# Arrange the plots in a grid layout and display them in the notebook.
output_notebook()
show(wnb_strain_plot)

Loading BokehJS ...

## Distributing Parameters

In [13]:
distribution_examples : Dict = {
	"Constant" : gf.Distribution(value=10, type_=gf.DistributionType.CONSTANT),
	"Uniform" : gf.Distribution(min_=5.0, max_=95.0, type_=gf.DistributionType.UNIFORM),
	"Normal" : gf.Distribution(mean=0.0, std=3.0, type_=gf.DistributionType.NORMAL),
	"Choice" : gf.Distribution(possible_values=["green", "red", "blue"], type_=gf.DistributionType.CHOICE),
	"Log" : gf.Distribution(min_=1.0, max_=10.0, type_=gf.DistributionType.LOG),
	"Powers of two": gf.Distribution(min_=2, max_=1024, type_=gf.DistributionType.POW_TWO, dtype=int)
}

In [14]:
for name, distribution in distribution_examples.items():
	print(f"{name} distribution samples: {distribution.sample(5)}")

Constant distribution samples: [10, 10, 10, 10, 10]
Uniform distribution samples: [74.85016178 25.14329199 55.87392786 26.81111317 85.09587743]
Normal distribution samples: [-1.88391588 -0.13734886 -0.83532383  1.49580309  2.27150535]
Choice distribution samples: ['red' 'blue' 'green' 'green' 'blue']
Log distribution samples: [8.02015636e+05 1.31988018e+06 1.81100060e+03 3.62379821e+06
 5.13799739e+08]
Powers of two distribution samples: [256, 128, 512, 2, 512]


In [15]:
mass_1_distribution_msun : gf.Distribution = gf.Distribution(
    min_=10.0, 
    max_=60.0, 
    type_=gf.DistributionType.UNIFORM
)
mass_2_distribution_msun : gf.Distribution = gf.Distribution(
    min_=10.0, 
    max_=60.0, 
    type_=gf.DistributionType.UNIFORM
)
inclination_distribution_radians: gf.Distribution = gf.Distribution(
    min_=0.0, 
    max_=np.pi, 
    type_=gf.DistributionType.UNIFORM
)

phenom_d_distribution_generator : gf.WaveformGenerator = gf.cuPhenomDGenerator(
    mass_1_msun=mass_1_distribution_msun,
    mass_2_msun=mass_2_distribution_msun,
    inclination_radians=inclination_distribution_radians
)

In [16]:
phenom_d_distribution_injection_generator : gf.InjectionGenerator = gf.InjectionGenerator(
	phenom_d_distribution_generator,
	parameters_to_return = [
		gf.WaveformParameters.MASS_1_MSUN, 
		gf.WaveformParameters.MASS_2_MSUN,
		gf.WaveformParameters.INCLINATION_RADIANS
	]
)

In [17]:
# Use the TensorFlow environment 'env' created earlier with gf.env()
with env:
    phenom_d_distributed_injections, _, phenom_d_distributed_parameters = next(
        phenom_d_distribution_injection_generator(num_examples_per_batch=4)
    )


In [18]:
mass_1_parameters : tf.Tensor = phenom_d_distributed_parameters[gf.WaveformParameters.MASS_1_MSUN]
mass_2_parameters : tf.Tensor = phenom_d_distributed_parameters[gf.WaveformParameters.MASS_2_MSUN]
inclination_parameters : tf.Tensor = phenom_d_distributed_parameters[gf.WaveformParameters.INCLINATION_RADIANS]

distributed_phenom_layout : List = []
for injection, mass_1, mass_2, inclination in zip(
		phenom_d_distributed_injections[0], 
		mass_1_parameters[0], 
		mass_2_parameters[0], 
		inclination_parameters[0]
	):
	distributed_phenom_layout.append([
		gf.generate_strain_plot(
			{
				"Plus Polarisation": injection[0],
				"Cross Polarisation": injection[1]
			},
			height=400,
			title=(
				"Phenom D Injection \n"
				f"Mass 1: {mass_1} M_sun \n"
				f"Mass 2: {mass_2} M_sun \n"
				f"Inclination: {inclination} Radians"
			)
		)
	])

# Arrange the plots in a grid layout and display them in the notebook.
grid = gridplot(distributed_phenom_layout)
output_notebook()
show(grid)

Loading BokehJS ...

## Generating Multiple Injections Simultaniously

In [19]:
duration_distribution_seconds : gf.Distribution = gf.Distribution(
    min_=0.1, 
    max_=0.9, 
    type_=gf.DistributionType.UNIFORM
)
min_frequency_distribution_hertz : gf.Distribution = gf.Distribution(
    min_=20.0, 
    max_=512.0, 
    type_=gf.DistributionType.UNIFORM
)
max_frequency_distribution_hertz: gf.Distribution = gf.Distribution(
    min_=20.0, 
    max_=512.0, 
    type_=gf.DistributionType.UNIFORM
)

wnb_distribution_generator : gf.WaveformGenerator = gf.WNBGenerator(
    duration_seconds=duration_distribution_seconds,
    min_frequency_hertz=min_frequency_distribution_hertz,
    max_frequency_hertz=max_frequency_distribution_hertz
)

In [20]:
multi_injection_generator : gf.InjectionGenerator = gf.InjectionGenerator(
	[phenom_d_distribution_generator, wnb_distribution_generator],
	parameters_to_return = [
		gf.WaveformParameters.MASS_1_MSUN, 
		gf.WaveformParameters.MASS_2_MSUN,
		gf.WaveformParameters.INCLINATION_RADIANS,
		gf.WaveformParameters.DURATION_SECONDS, 
		gf.WaveformParameters.MIN_FREQUENCY_HERTZ,
		gf.WaveformParameters.MAX_FREQUENCY_HERTZ,

	]
)

In [21]:
# Use the TensorFlow environment 'env' created earlier with gf.env()
with env:
    multi_injections, _, multi_parameters = next(multi_injection_generator(num_examples_per_batch=4))

2024-01-12 01:41:39.772263: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'num_waveforms' with dtype int32
	 [[{{node num_waveforms}}]]


In [22]:
mass_1_parameters : tf.Tensor = multi_parameters[gf.WaveformParameters.MASS_1_MSUN][0]
mass_2_parameters : tf.Tensor = multi_parameters[gf.WaveformParameters.MASS_2_MSUN][0]
inclination_parameters : tf.Tensor = multi_parameters[gf.WaveformParameters.INCLINATION_RADIANS][0]
duration_parameters : tf.Tensor = multi_parameters[gf.WaveformParameters.DURATION_SECONDS][1]
min_frequency_parameters : tf.Tensor = multi_parameters[gf.WaveformParameters.MIN_FREQUENCY_HERTZ][1]
max_frequency_parameters : tf.Tensor = multi_parameters[gf.WaveformParameters.MAX_FREQUENCY_HERTZ][1]

distributed_phenom_layout : List = []
for phenom_d_injections, wnb_injections, mass_1, mass_2, inclination, duration, min_freq, max_freq in zip(
		multi_injections[0], 
		multi_injections[1],
		mass_1_parameters, 
		mass_2_parameters, 
		inclination_parameters,
		duration_parameters,
		min_frequency_parameters,
		max_frequency_parameters
	):
	distributed_phenom_layout.append([
		gf.generate_strain_plot(
			{
				"Plus Polarisation": phenom_d_injections[0],
				"Cross Polarisation": phenom_d_injections[1]
			},
			height=400,
			width=500,
			title=(
				"Phenom D Injection \n"
				f"Mass 1: {mass_1} M_sun \n"
				f"Mass 2: {mass_2} M_sun \n"
				f"Inclination: {inclination} Radians"
			)
		),
		gf.generate_strain_plot(
			{
				"Plus Polarisation": wnb_injections[0],
				"Cross Polarisation": wnb_injections[1]
			},
			height=400,
			width=500,
			title=(
				"WNB Injection \n"
				f"Duration: {duration} S \n"
				f"Min Frequency: {min_freq} Hz \n"
				f"Inclination: {max_freq} Radians"
			)
		)
	])

# Arrange the plots in a grid layout and display them in the notebook.
grid = gridplot(distributed_phenom_layout)
output_notebook()
show(grid)

Loading BokehJS ...

## Loading Injections from Config JSON


## Varying Injection Chance